##Prep

In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

In [11]:
#mount my drive
from google.colab import drive
drive.mount('/content/drive')

# Get the path to the CSV file on Drive
csv_path = "/content/drive/MyDrive/Data Mining/Project/Data/final_merged.csv"

# Read the CSV file into a Pandas DataFrame
final = pd.read_csv(csv_path)

#drop duplicates
final = final.drop_duplicates()

# Reset the index to create a new integer index
final.reset_index(drop=True, inplace=True)

# Drop the 'msno' column if it's no longer needed
final.drop('msno', axis=1, inplace=True)

# Rename the new index
final.index.name = 'msno'

final.reset_index(inplace=True)

## Feature Engineering

In [7]:
cutoff_date = pd.to_datetime('2017-03-31')

# Assuming your DataFrame is named `final`
# Convert columns to datetime if they are not already
final['registration_init_date'] = pd.to_datetime(final['registration_init_year'].astype(int).astype(str) + '-' +
                                                 final['registration_init_month'].astype(int).astype(str) + '-' +
                                                 final['registration_init_date'].astype(int).astype(str))

final['transaction_date'] = pd.to_datetime(final['transaction_date_year'].astype(int).astype(str) + '-' +
                                           final['transaction_date_month'].astype(int).astype(str) + '-' +
                                           final['transaction_date_date'].astype(int).astype(str))

final['membership_expire_date'] = pd.to_datetime(final['membership_expire_date_year'].astype(int).astype(str) + '-' +
                                                       final['membership_expire_date_month'].astype(int).astype(str) + '-' +
                                                       final['membership_expire_date_date'].astype(int).astype(str))

# Compute time until membership expiration from transaction date (in days)
final['days_until_membership_expiration'] = (final['membership_expire_date'] - final['transaction_date']).dt.days

# Extract seasonal features from transaction dates
final['transaction_quarter'] = final['transaction_date'].dt.quarter

# Compute age of account at the cutoff date (in days)
final['account_age_days_at_cutoff'] = (cutoff_date - final['registration_init_date']).dt.days

final= final.drop(columns=['Unnamed: 0'],axis=1)

final= final.drop(columns=['is_cancel_sum'],axis=1)

datetime_cols = [col for col in final.columns if final[col].dtype == 'datetime64[ns]']

final.drop(datetime_cols, axis=1, inplace=True)

##Encoding

In [8]:
# Apply one-hot encoding to the specified columns

final_encoded = pd.get_dummies(final, columns=['city', 'registered_via', 'payment_method_id'],
                               prefix=['city', 'registered_via', 'payment_method_id'])



#Scaling

In [9]:
# set 'msno' as index column
final_encoded.set_index('msno', inplace=True)

# Selecting numerical columns that need to be scaled
columns_to_scale = final_encoded.select_dtypes(include=['int64', 'float64','int32']).columns.drop(['is_churn'])

final_scaled=final_encoded.copy()

scaler = StandardScaler()
final_scaled[columns_to_scale] = scaler.fit_transform(final_scaled[columns_to_scale])

In [10]:
# prompt: export final df as a csv file and store it to my google Drive

final_encoded.to_csv('final_scaled.csv')
!cp final_scaled.csv "/content/drive/MyDrive/Data Mining/Project/Data/"